## Using QGIS and Support Vector Machines to Differentiate Species of Meadow Jumping Mice (*Zapus hudsonius*) and Western Jumping Mice (*Zapus princeps*)
**Matthew Clark**

**Introduction**

Colorado is home to two different jumping mouse species, the meadow jumping mouse (*Zapus hudsonius*) and the western jumping mouse (*Zapus princeps*). Many of the subspecies of meadow jumping mice including the Preble's meadow jumping mouse (*Zapus hudsonius preblei*) are listed as endangered (US Fish and Wildlife Service, 2021). Conservation plans for construction along the Front Range frequently have to include plans to avoid or minimize negative effects on the jumping mice. In contrast, the western jumping mouse is thriving. This project will look into whether habitat differences between the two species can be classified by a machine learning algorithm. This will take the form of a support vector machine learner. The goal of the project is to specifically see if the habitat characteristics of meadow jumping mouse habitat can be detected by the SVM. For this reason, the sightings of meadow  jumping mice and western jumping mice will be compared to see if the model can tell the differences between them. The US Fish and Wildlife Service recommends that habitat for the Preble's meadow jumping mouse be within 110 meters of a water body, be it a stream, river, pond or lake (Trainor et. al. 2012). However, Trainor et. al. (2012) have found that some mice can be found as far as 340 meters from a body of water as the mice also frequent the grasslands in the vacinity of rivers. These parameters would be important in this project. 

**Data Sources**

Data was acquired from BISON (Biodiversity Information Serving Our Nation) in the form of point shapefiles containing data on the geographic location of the specimens, the institutions who collected the data, taxonomic information and the date in which it was collected (BISON, n.d.). All data points before 1990 were excluded as the areas where this data was found may have been developed by the present day. In addition the data was clipped to just include data points from Colorado. Institutions who contributed to the data include the Denver Museum of Nature and Science, NatureServe Network, the Museum of Southwestern Biology, Fort Hayes Sternberg Museum of Natural History, University of Alaska Museum of the North, iNaturalist.org, Angelo State Natural History Museum, Charles R. Conner Museum and the University of Colorado Museum of Natural History.

Data for land cover was retrieved from the US Geological Survey's *2011 National Land Cover Dataset* or NLDS 2011 (United States Geological Survey, 2011). This dataset contains 20 different land cover types. This study included open water, open space, developed areas (low, medium and high), barren ground, deciduous forests, coniferous forests, mixed forests, shrubs, grasslands, pasture, agricultural areas, wooded wetlands and herbaceous emergent grasslands. Data for rivers was acquired from the USGS National Geospatial Program's map, *NHD 20200615 for Colorado State or Territory SDshapefile Model Version 2.2.1* (US Geological Survey, 2020). Elevation data was collected from a dataset created by ColoradoView/UV-B Monitoring and Research (n.d.). This dataset consisted of 28 separate raster files representing a digital elevation model (DEM) for the state of Colorado.

**QGIS**

QGIS software was used to process the data needed for the project. This is the key way in which this project was different from others in that data was not collected via an API, but rather by combining external data sources using GIS software. QGIS is an open source software in comparison to ArcGIS which is very expensive. QGIS allowed the data to be processed remotely without an expensive subscription to ArcGIS. At the beginning of the project, a proof-of-concept model was created to see if it was indeed possible to create the required data on QGIS. 

All data needed to be converted to the North America Lambert Conformal Conic projection to ensure that the data was able to line up and overlap properly. A dataset of Colorado Counties was used as a mask to clip the 2011 National Land Cover Database (NLDS 2011) to just Colorado. In the case of the proof-of-concept model, this was just to Douglas County. The river data from the USGS was clipped to just include Colorado counties, as it previously included the entire watersheds in the region around Colorado. This data was composed not just of major rivers, but also the flowlines in each direction (North, South, East, Northeast, Southeast, West, Northwest and Southwest). In order to calculate the distance from streams required this data to be merged. This data was then transformed into a raster using the *rasterize* function. The finest resolution possible was a 10-square meter resolution. Distances from rivers were then calculated using 10-square meter increments using the *proximity* function.

The BISON data included sightings of both the western and meadow jumping mice, and these shapefiles needed to be converted to the Lambert Conformal Conic coordinate system and then combined using the *merge* function. Because the datasets all contained the same columns, this was actually very easy. Buffers were calculated at a distance of  340-meters as per the observations of Trainor et. al. 2012. The *zonal histogram* function was then used to perform a count of the number of pixels in each land cover type that overlapped with each buffer polygon. These values came from approximately 28 square-meter cells that would later be multiplied by 28 with Pandas to produce the area in square-meters of each habitat type within each buffer. The *zonal statistics* function was used to find the average distance value in 10x10 square meter pixels from a river within each buffer. This value would later be multiplied by 10 to estimate the average distance in square meters from rivers within the buffer. The zonal statistics function would also perform a similar operation to determine the average elevation in meters within each buffer.



### Modules used in this project

**Pandas**

**numpy**

**seaborn**

**numpy**

**IPython**

**Matplotlib**

**sklearn**
sklearn functions include train_test_split, accuracy_score, svm, metrics, cross_val_score, DecisionTreeClassifier, FactorAnalysis

In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np
from IPython.display import display
from IPython.display import Image
import seaborn as sns
import matplotlib.pyplot as plt
# Suppress depreciation warnings 
import warnings 
warnings.filterwarnings('ignore')
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import FactorAnalysis

**Preparing the Data**

Data preparation involved exporting the data from the final data's attribute table as a .csv file, and then using Pandas to edit the data. This involved creating a list of new names for the subsequent columns in the dataset and applying them with the .columns function. As the data was prepared in QGIS, it was not necessary to fill in columns with missing data. Lambda functions were then used to further edit the data. Some points fell outside of the scope of the NLCD and had a value of NoData in some or all of their buffers. To fix this problem, these cells were multiplied by zero in the lambda functions so that this column would not interfere with data analysis. Each column for land cover in the NLCD had counts of approximately 28 square meter cells, so each column needed to be multiplied by 28. The finest resolution possible for the distance from rivers were 10 square meter cells, so the values in this column needed to be multiplied by 10. 

In [2]:
df = pd.read_csv('C:/Users/Matthew/MyDocuments/New_Practicum_Stats.csv')

In [3]:
df.columns = ['fid', 'bisonID', 'Species', 'xcoord', 'ycoord', 'NoData', 'Open_water', 'Dev_open_space', 'Dev_low', 'Dev_medium', 'Dev_high', 'Barren_land', 'Deci_forest', 'Conifer_forest', 'Mixed_forest', 'Shrubland', 'Grassland', 'Pasture', 'Agriculture', 'Wetlands_woody', 'Wetlands_herb', 'River_Distance', 'Elevation']
print(df.columns)

Index(['fid', 'bisonID', 'Species', 'xcoord', 'ycoord', 'NoData', 'Open_water',
       'Dev_open_space', 'Dev_low', 'Dev_medium', 'Dev_high', 'Barren_land',
       'Deci_forest', 'Conifer_forest', 'Mixed_forest', 'Shrubland',
       'Grassland', 'Pasture', 'Agriculture', 'Wetlands_woody',
       'Wetlands_herb', 'River_Distance', 'Elevation'],
      dtype='object')


In [4]:
# Using Lambda functions to covert data into square meters
df['NoData'] = df['NoData'].apply(lambda x: x*0)
df['Open_water'] = df['Open_water'].apply(lambda x: x*28)
df['Dev_open_space'] = df['Dev_open_space'].apply(lambda x: x*28)
df['Dev_low'] = df['Dev_low'].apply(lambda x: x*28)
df['Dev_medium'] = df['Dev_medium'].apply(lambda x: x*28)
df['Dev_high'] = df['Dev_high'].apply(lambda x: x*28)
df['Barren_land'] = df['Barren_land'].apply(lambda x: x*28)
df['Deci_forest'] = df['Deci_forest'].apply(lambda x: x*28)
df['Conifer_forest'] = df['Conifer_forest'].apply(lambda x: x*28)
df['Mixed_forest'] = df['Mixed_forest'].apply(lambda x: x*28)
df['Shrubland'] = df['Shrubland'].apply(lambda x: x*28)
df['Grassland'] = df['Grassland'].apply(lambda x: x*28)
df['Pasture'] = df['Pasture'].apply(lambda x: x*28)
df['Wetlands_woody'] = df['Wetlands_woody'].apply(lambda x: x*28)
df['Wetlands_herb'] = df['Wetlands_herb'].apply(lambda x: x*28)
df['River_Distance'] = df['River_Distance'].apply(lambda x: x*10)

**Coding for Data Exploration**

Data exploration was done with the Seaborn, Matplotlib and IPython modules. Creating these plots relied on seperating numerical data for land cover types, elevations and river distances using the .iloc[] function from Pandas. The pair plot was created Seaborn's pairplot() function. In contrast, the heat map involved converting the dataframe into an array, and then using Seaborn's heatmap function combined with a corr() function using techniques from Anita (2019). 

In [5]:
gdf = df.drop(['fid', 'bisonID', 'xcoord', 'ycoord', 'NoData', 'Open_water', 'Elevation', 'River_Distance'], axis = 1)
gdf

,Species,Dev_open_space,Dev_low,Dev_medium,Dev_high,Barren_land,Deci_forest,Conifer_forest,Mixed_forest,Shrubland,Grassland,Pasture,Agriculture,Wetlands_woody,Wetlands_herb
0,Zapus princeps,0,0,0,0,0,3948,7364,0,1008,0,0,0,0,0
1,Zapus princeps,0,0,0,0,0,2156,336,0,3248,0,3696,0,2436,0
2,Zapus princeps,0,0,0,0,0,2156,336,0,3248,0,3696,0,2436,0
3,Zapus princeps,0,0,0,0,0,2156,336,0,3248,0,3696,0,2436,0
4,Zapus princeps,0,0,0,0,0,2324,616,0,5236,0,1512,0,2324,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,Zapus hudsonius,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0
398,Zapus hudsonius,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0
399,Zapus hudsonius,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0
400,Zapus hudsonius,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0


In [6]:
pair_m = df.iloc[:, 6:23]

In [7]:
pair_m

,Open_water,Dev_open_space,Dev_low,Dev_medium,Dev_high,Barren_land,Deci_forest,Conifer_forest,Mixed_forest,Shrubland,Grassland,Pasture,Agriculture,Wetlands_woody,Wetlands_herb,River_Distance,Elevation
0,0,0,0,0,0,0,3948,7364,0,1008,0,0,0,0,0,184.628104,0.000000
1,0,0,0,0,0,0,2156,336,0,3248,0,3696,0,2436,0,130.768821,1362.505689
2,0,0,0,0,0,0,2156,336,0,3248,0,3696,0,2436,0,130.768821,1362.505689
3,0,0,0,0,0,0,2156,336,0,3248,0,3696,0,2436,0,130.768821,1362.505689
4,0,0,0,0,0,0,2324,616,0,5236,0,1512,0,2324,0,200.208428,1372.888781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0,136.683588,1761.247582
398,0,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0,136.683588,1761.247582
399,0,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0,136.683588,1761.247582
400,0,1008,0,0,0,0,0,0,0,2688,6916,0,0,1624,0,136.683588,1761.247582


**Results for Data Explortation**

The shear number of variables made interpreting the pairplot difficult. Many variables had skewed distributions due to the high number of zero values. It is hoped that the large sample sizes will be sufficient to balance for this effect. Strong relationships between the different development types were detected in both the correlation matrix with a heatmap and with the pairplot. The heat map in particular showed a strong relationship between open space development and low developed areas, medium development and high development areas and between river distance and grasslands. The final correlation is likely due to the fact that many Preble's jumping mouse sightings were found in grasslands within 320 meters from water, as observed in Trainor et. al. (2012). A correlation between mixed forest and decidous forest was also observed. Coniferous forest correlated heavily in a positive way to elevation and strongly in a negative way to grasslands. This is not surprising as the two habitat types are found at different elevations. 

**Creating the SVM model and Displaying the Results**

The actual SVM model was created using code from the YouTube channel CMS WisCon (April 30, 2020). This source was chosen because this was the first time I created an SVM using Python, and I was having difficulty creating the model from a Pandas dataframe. The first step involved creating a random seed for the model using numpy's random.seed() function. Then the model was converted to an array using the .values function. Testing and training datasets were created from the original dataframe. The training data including values for the land area in square meters of different habitat types, the distance in rivers in meters and the elevation of the habitat. Latitude and longitude were left out due to the great difference in orders of magnitude of the data. The colunm for species was used for the dependent variable as it contained the species of the mouse included in the data point. The train_test_split() function from sklearn was used to split both the independent variable and the labels for the dependent variable into train and testing sets. The SVC() from sklearn was used to construct the support vector machine. Versions of the SVM were also created with linear, polynomial and radial basis function kernals. The models were fit with the .fit function, and testd using the .predict() function. The accuracy of the model was printed using the accuracy_score() function. The results of the data were displayed with a confusion matrix created from code by website Edpresso (2021) and sklearn's metrics package. These included a confusion matrix (metrics.confusion_matrix) and a classification report with precision, recall, f1-scores and support (metrics.classification_report).  Results from the support vectors machines were graphed in dataframes created using Python and R Tips (2018).

In [8]:
# need to convert pandas data frame into an array (CMS WisCon, April 30, 2020)
m_array = df.values

**Generic SVM**

The first model to be tested was a generic SVM model with default settings. 

In [9]:
# Training and testing splitting the data (CMS WisCon, April 30, 2020)
np.random.seed(250)
x = m_array[:,6:23]
y = m_array[:,2]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [10]:
# Training the support vector classifier (CMS WisCon, April 30, 2020)
model = SVC()
model.fit(x_train, y_train)

SVC()

In [11]:
# Testing the model's accuracy (CMS WisCon, April 30, 2020)
predict_mouse = model.predict(x_test)
print("Accuracy: ", accuracy_score(y_test, predict_mouse))

Accuracy:  0.8925619834710744


In [12]:
# Printing a confusion matrix
# Edpresso (2021) How to create a confusion matrix in Python using scikit-learn [Blog] Retrieved from: https://www.educative.io/edpresso/how-to-create-a-confusion-matrix-in-python-using-scikit-learn
print(metrics.confusion_matrix(y_test, predict_mouse, labels=["Zapus hudsonius", "Zapus princeps"]))

[[69  5]
 [ 8 39]]


In [13]:
# Creating a data frame of the confusion matrix
# Code from Python and R Tips(2018)
gsvm = metrics.confusion_matrix(y_test, predict_mouse, labels=["Zapus hudsonius", "Zapus princeps"])
gsvm = pd.DataFrame(gsvm)
gsvm.columns = ['Z.hudsonius', 'Z.princeps']
gsvm

,Z.hudsonius,Z.princeps
0,69,5
1,8,39


In [14]:
# Printing a classification report
print(metrics.classification_report(y_test, predict_mouse, labels=["Zapus hudsonius", "Zapus princeps"]))

                 precision    recall  f1-score   support

Zapus hudsonius       0.90      0.93      0.91        74
 Zapus princeps       0.89      0.83      0.86        47

       accuracy                           0.89       121
      macro avg       0.89      0.88      0.89       121
   weighted avg       0.89      0.89      0.89       121



In [15]:
# Calculating the kappa statistic
ksvm = sklearn.metrics.cohen_kappa_score(y_test, predict_mouse, labels=["Zapus hudsonius", "Zapus princeps"])
ksvm

0.7712

In [16]:
# Creating a dataframe of the results
# Code from Python and R Tips(2018)
Metrics_g = ['Accuracy', 'Precision', 'Recall', 'f1-score', 'Kappa']
results = [0.893, 0.89, 0.88, 0.89, 0.77]
svmr = {'Metric': Metrics_g, 'macro_avg':results}
svmr = pd.DataFrame(svmr, columns=['Metric', 'macro_avg'])
svmr

,Metric,macro_avg
0,Accuracy,0.893
1,Precision,0.890
2,Recall,0.880
3,f1-score,0.890
4,Kappa,0.770


In [17]:
# Removing variables with high correlation
df_slimmer = df.drop(['Dev_low', 'Dev_medium', 'Mixed_forest', 'Elevation', 'River_Distance'], axis = 1)

In [18]:
# Converting pandas data frame into an array (CMS WisCon, April 30, 2020)
slimmer_array = df_slimmer.values

In [19]:
# Training and testing split (CMS WisCon, April 30, 2020)
np.random.seed(250)
x = slimmer_array[:,6:18]
y = slimmer_array[:,2]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

**Decision Tree Models**

In [20]:
# Creating the decision tree
# Raschka and Mirjalili (2019) pg. 96
tree_mouse = DecisionTreeClassifier(criterion='gini',
                                   max_depth=4, 
                                   random_state=1)
tree_mouse.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=4, random_state=1)

In [21]:
# Testing the slimmed down model
predict_tm = tree_mouse.predict(x_test)
print("Accuracy: ", accuracy_score(y_test, predict_tm))

Accuracy:  0.9173553719008265


In [22]:
# Printing a confusion matrix
# https://www.educative.io/edpresso/how-to-create-a-confusion-matrix-in-python-using-scikit-learn
print(metrics.confusion_matrix(y_test, predict_tm, labels=["Zapus hudsonius", "Zapus princeps"]))

[[73  1]
 [ 9 38]]


In [23]:
# Code from Python and R Tips(2018)
tmcm = metrics.confusion_matrix(y_test, predict_tm, labels=["Zapus hudsonius", "Zapus princeps"])
tmdf = pd.DataFrame(tmcm)
tmdf.columns = ['Z.hudsonius', 'Z.princeps']
tmdf

,Z.hudsonius,Z.princeps
0,73,1
1,9,38


In [24]:
# Creating a classificaiton report
print(metrics.classification_report(y_test, predict_tm, labels=["Zapus hudsonius", "Zapus princeps"]))

                 precision    recall  f1-score   support

Zapus hudsonius       0.89      0.99      0.94        74
 Zapus princeps       0.97      0.81      0.88        47

       accuracy                           0.92       121
      macro avg       0.93      0.90      0.91       121
   weighted avg       0.92      0.92      0.92       121



In [25]:
# Calculating the Cohen's Kappa Score
ksvm = sklearn.metrics.cohen_kappa_score(y_test, predict_tm, labels=["Zapus hudsonius", "Zapus princeps"])
ksvm

0.8204747774480712

In [26]:
# Creating a data frame of the results
# Code from Python and R Tips(2018)
Metrics = ['Accuracy', 'Precision', 'Recall', 'f1-score', 'kappa']
results = [0.901, 0.93, 0.90, 0.91, 0.820]
ksvmr = {'Metric': Metrics, 'macro_avg':results}
ksvmr = pd.DataFrame(ksvmr, columns=['Metric', 'macro_avg'])
ksvmr

,Metric,macro_avg
0,Accuracy,0.901
1,Precision,0.930
2,Recall,0.900
3,f1-score,0.910
4,kappa,0.820


**Bibliography**

US Fish and Wildlife Service (January 6th, 2021) Preble’s Meadow Jumping Mouse Retrieved from: https://www.fws.gov/mountain-prairie/es/preblesMeadowJumpingMouse.php

Trainor, Anne M., Shenk, Tanya M. and Wilson, Kenneth R. (2012) Spatial, temporal, and biological factors associated with Prebles meadow jumping mouse (Zapus hudsonius preblei) home range. Journal of Mammalogy. 93(2), pgs. 429-438. Doi: 10.1644/11-MAMM-A-049.1

Python and R Tips (January 10, 2018) How to Create Pandas Dataframe from Multiple Lists? Pandas Tutorial. [Blog] Retrieved from: https://cmdlinetips.com/2018/01/how-to-create-pandas-dataframe-from-multiple-lists/

CMS WisCon(April 30, 2020) SVM Classifier in Python on Real Data Set [YouTube] Retrieved from: https://www.youtube.com/watch?v=Vv5U0kjYebM

Edpresso (2021) How to create a confusion matrix in Python using scikit-learn [Blog] Retrieved from: https://www.educative.io/edpresso/how-to-create-a-confusion-matrix-in-python-using-scikit-learn

Anita, Okoh (August 20th, 2019) Seaborn Heatmaps: 13 Ways to Customize Correlation Matrix Visualizations [Heartbeat] Retrieved from: https://heartbeat.fritz.ai/seaborn-heatmaps-13-ways-to-customize-correlation-matrix-visualizations-f1c49c816f07

scikit-learn developers (2007-2020) 3.1. Cross-validation: evaluating estimator performance. Retrieved from: https://scikit-learn.org/stable/modules/cross_validation.html

Boyle, Tara (February 3rd, 2019) *Dealing with Imbalanced Data: A guide to effectivly handling imbalanced datasets in Python*. [Towards Data Science] Retrieved from: https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18

Raschka, Sebastian and Vahid Mirjalili (2019) *Python Machine Learning Third Edition*. Packt Publishing Ltd.

Chen, Daniel y. (2018) *Pandas for Everyone: Python Data Analysis*. Pearson Education, Inc. 

Lantz, Brett () *Machine Learning with R: Second Edition*. Packt Publishing Ltd.

DMNS Mammal Collection (Arctos). Denver Museum of Nature and Science. Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

NatureServe Network Species Occurrence Data. Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

Museum of Southwestern Biology. Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

Fort Hayes Sternberg Museum of Natural History. Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

University of Alaska Museum of the North. Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021 

iNaturalist.org.Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

Angelo State Natural History Museum (ASNHC).Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

Charles R. Conner Museum.Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

University of Colorado Museum of Natural History.Accessed through Biodiversity Information Serving Our Nation (BISON) (n.d.) Zapus hudsonius prebli [Data File] Retrieved from https://bison.usgs.gov/#home on 1/16/2021

US Geological Survey. US Department of the Interior. (2014). National Land Cover Database 2011 (NLCD2011). [Raster]. Multi-Resolution Land Characteristics Consortium (MRLC). Retrieved from https://www.mrlc.gov/nlcd11_data.php on April 4th 2017. 

CDPHE_user_commuity, Colorado Department of Public Health and the Environment (2/19/2018) Colorado County Boundaries [Data File] Retrieved from: https://data-cdphe.opendata.arcgis.com/datasets/colorado-county-boundaries on 1/16/2021

U.S. Geological Survey, National Geospatial Program (06/15/2020) Retrieved from: NHD 20200615 for Colorado State or Territory Shapefile Model Version 2.2.1 [Data File] https://viewer.nationalmap.gov/basic/?basemap=b1&category=nhd&title=NHD%20View#/ on 1/26/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 1-1. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 1-2. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 1-3. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 1-4. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 1-5. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 1-6. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 1-7. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 2-1. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 2-2. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 2-3. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 2-4. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 2-5. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 2-6. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 2-7. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 3-1. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 3-2. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 3-3. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 3-4. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 3-5. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 3-6. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 3-7. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 4-1. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 4-2. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 4-3. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 4-4. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 4-5. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 4-6. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021

ColoradoView/UV-B Monitoring and Research (n.d.) Colorado Digital Elevation Model files - 1 degree: Section 4-7. [Data File] Retrieved from: https://www.coloradoview.org/aerial-imagery/ on 2/06/2021





